In [0]:
%run ../landing-layer/01-config

In [0]:
config = config()
df = config.create_config_df()
df.display()

In [0]:
dbutils.widgets.text('hospital_name','hos-a')

In [0]:
hospital = dbutils.widgets.get('hospital_name')
print(hospital)

In [0]:
class copy_sql_bronze():
    def __init__(self):
        self.number_of_rows_copied = 0
    def full_data(self,datasource,tablename):
        print(tablename)
        temp_df = spark.read.format('csv')\
            .option('header','true')\
            .option('inferschema','true')\
            .option('path',config.get_base_dir()+'/'+tablename+'.csv')\
            .load()
        print(temp_df.columns)
        self.number_of_rows_copied = temp_df.count()
        temp_df.write.format('parquet')\
            .mode('overwrite')\
            .save(config.get_bronze_volume_dir()+'/'+tablename+'/'+datasource)
        print('Full Load Completed...')

    def incremental_data(self,tablename,watermarkcolumnname):
        from pyspark.sql.functions import cast
        last_insertion_date = spark.sql(f'''
                        SELECT coalesce(MAX(loaddate),'1999-12-31') from healthcare_proj_privatews.audit.load_logs where tablename='{tablename}'  ''').collect()[0][0]
        print(last_insertion_date)
        temp_df = spark.read.format('csv')\
            .option('header','true')\
            .option('mode','PERMISSIVE')\
            .option('inferschema','true')\
            .option('path',config.get_base_dir()+'/'+tablename+'.csv')\
            .load()
        
        temp_df = temp_df.filter(col(watermarkcolumnname) >=last_insertion_date)
        self.number_of_rows_copied = temp_df.count()
        temp_df.write.format('parquet')\
            .mode('append')\
            .save(config.get_bronze_volume_dir()+'/'+tablename)
        print('Incremental Load Completed...')


    def insert_audit_logs(self,datasource,tablename,watermarkcolumnname):
        
        from pyspark.sql.functions import current_timestamp
        from datetime import datetime
        current_time = datetime.now()
        # print(datasource,tablename,watermarkcolumnname,current_time)
        spark.sql('CREATE SCHEMA IF NOT EXISTS healthcare_proj_privatews.audit');
        spark.sql('''CREATE TABLE IF NOT EXISTS 
                  healthcare_proj_privatews.audit.load_logs
                  (
                      data_source STRING,
                      tablename   STRING,
                      numberofrowscopied BIGINT,
                      watermarkcolumnname STRING,
                      loaddate TIMESTAMP
                  )
                  ''');
        spark.sql( f'''
        INSERT INTO healthcare_proj_privatews.audit.load_logs
        VALUES ('{datasource}', '{tablename}', {self.number_of_rows_copied}, '{watermarkcolumnname}', '{current_time}')
    ''')
        

In [0]:
copy_sql_bronze = copy_sql_bronze()

In [0]:
df.display()

In [0]:
from pyspark.sql.functions import col, array
df = df.filter(col('datasource')==hospital)
for i in df.select('tablename').collect():
    tablename = i[0].split('.')[1]
    loadtype = df.filter(col('tablename')==i[0]).select('loadtype').collect()[0][0]
    datasource = df.filter(col('tablename')==i[0]).select('datasource').collect()[0][0]
    watermark = df.filter(col('tablename')==i[0]).select('watermark').collect()[0][0]
    
    if loadtype == 'Incremental':
        # copy_sql_bronze.incremental_data(tablename,watermark)
        copy_sql_bronze.full_data(datasource,tablename)
        copy_sql_bronze.insert_audit_logs(datasource,tablename,watermark)

    if loadtype == 'Full':
        copy_sql_bronze.full_data(datasource,tablename)
        copy_sql_bronze.insert_audit_logs(datasource,tablename,watermark)
